In [1]:
import pickle
import pandas as pd

In [2]:
eval_traffic = "/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/evaluation_datasets/V2_eval_trafficDataset_2024-3-16_1753hours_1296000steps.csv"
eval_vehicle = "/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/evaluation_datasets/V2_eval_vehicleDataset_2024-3-16_1753hours_1296000steps.csv"
model_path = '/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/xgboost/saved_models/V24/xg_model.pkl'
NUM_OF_JUNCTIONS = 8
ROLLING_WINDOW_LENGTH = 600
TRAVEL_TIME_WINDOW = 600
LENGTH_OF_SIM = 1296000
generate_incidents_csv = True

In [3]:
print(f"Number of junctions considered for dataset prep: {NUM_OF_JUNCTIONS}")
print(f"Rolling window length considered for dataset prep: {ROLLING_WINDOW_LENGTH}")
print(f"Travel time window length considered for dataset prep: {TRAVEL_TIME_WINDOW}")

Number of junctions considered for dataset prep: 8
Rolling window length considered for dataset prep: 600
Travel time window length considered for dataset prep: 600


In [4]:
dtype = {
    'step' : 'int64',
    'time_of_day' : 'int64',
    'identified_edge' : 'object',
    'junction_mean_speed' : 'float64',
    'traffic_count' : 'float64',
    'traffic_occupancy' : 'float64',
    'vehicles_per_lane_1' : 'int64',
    'vehicles_per_lane_0' : 'int64',
    'lane_mean_speed_0' : 'float64',
    'lane_mean_speed_1' : 'float64',
    'incident_edge': 'object',
    'incident_start_time': 'float64',
    'incident_type': 'object',
    'accident_label': 'bool',
    'accident_id': 'object',
    'accident_duration': 'float64',
    'incident_lane': 'object'
}
df_traffic = pd.read_csv(eval_traffic,dtype=dtype)

In [5]:
df_traffic.head()

,step,time_of_day,identified_edge,junction_mean_speed,traffic_count,traffic_occupancy,vehicles_per_lane_1,vehicles_per_lane_0,lane_mean_speed_0,lane_mean_speed_1,incident_edge,incident_start_time,incident_type,accident_label,accident_id,accident_duration,incident_lane,junction
0,0,0,533573776#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
1,0,0,436794680#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
2,0,0,256917837#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
3,0,0,436791498#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
4,0,0,436794701#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1


In [6]:
INCOMING_EDGES = ['436794670','436791113#0','436794680#0','436940278','-643913497','436943745#0','351673438','-613687451#1','436943750#0','436943762#0','-436942356#1','436942362#0','436789580#1','436942357','436794669','1051038541#0','-436794679#3','533371302#0','436942374','436790491','-436942381#3']

OUTGOING_EDGES = ['533573776#0','5607328#0','519448767','436940270','-436943745#2','436943742','436943774','436943743#0','-436943762#2','436942356#0','-436942362#3','436789564#0','531969915#0','436794679#0','-1088637809#1','436942385#0','-436794676#1','436790495','-1033824750','30031286#0','436942381#0']


In [7]:
sensors = INCOMING_EDGES+OUTGOING_EDGES
print(len(sensors))

42


In [8]:
sensors_more = {    1:["533573776#0","436794680#0","256917837#0","436791498#0","436794701#0","-436794701#3","5607328#0","436791113#0"],
    2:["436794679#0","-436794679#3","436942387#0","436942371#0","-1088637809#1","436794670","436942385#0","1051038541#0"],
    3:["436942349#0","-436942349#3","-403117287#3","403117287#0","-436794676#1","436794669","531969915#0","436942357"],
    5:["30031286#0","436790491","436942381#0","-436942381#3","436790495","533371302#0","-1033824750","436942374"],
    6:["422264711#0","422264712#0","-436942362#3","436942362#0","436789564#0","436789580#1","436942356#0","-436942356#1"],
    7:["436940270","-643913497","519448767","436940278","436940284","-597312446","422267251","422267252"],
    8:["436943742","351673438","436943774","-613687451#1","-436943745#2","436943745#0","-436943776#4","436943776#0"],
    9:["406379332","436943746#0","-436943762#2","436943762#0","436943743#0","436943750#0","436943757#0","-436943757#1"]
}

excess_sensors = []
for key in sensors_more.keys():
    
    excess_sensors = excess_sensors + sensors_more[key]

In [9]:
set_diff = set(excess_sensors).difference(set(sensors))
print(set_diff)
print(len(set_diff))

{'436794701#0', '-436943776#4', '-403117287#3', '-436942349#3', '422267252', '436943776#0', '436942371#0', '-597312446', '436942387#0', '406379332', '436943757#0', '-436943757#1', '403117287#0', '436791498#0', '-436794701#3', '422264712#0', '256917837#0', '436942349#0', '436943746#0', '422264711#0', '422267251', '436940284'}
22


In [10]:
excess_list = list(set_diff)
print(excess_list)

['436794701#0', '-436943776#4', '-403117287#3', '-436942349#3', '422267252', '436943776#0', '436942371#0', '-597312446', '436942387#0', '406379332', '436943757#0', '-436943757#1', '403117287#0', '436791498#0', '-436794701#3', '422264712#0', '256917837#0', '436942349#0', '436943746#0', '422264711#0', '422267251', '436940284']


In [11]:
df_traffic_reduced = df_traffic[~df_traffic['identified_edge'].isin(excess_list)]

In [12]:
df_traffic_reduced.head()

,step,time_of_day,identified_edge,junction_mean_speed,traffic_count,traffic_occupancy,vehicles_per_lane_1,vehicles_per_lane_0,lane_mean_speed_0,lane_mean_speed_1,incident_edge,incident_start_time,incident_type,accident_label,accident_id,accident_duration,incident_lane,junction
0,0,0,533573776#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
1,0,0,436794680#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
6,0,0,5607328#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
7,0,0,436791113#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
8,0,0,436794679#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,2


In [13]:
df_traffic.shape

(82944000, 18)

In [14]:
df_traffic_reduced.shape

(54432000, 18)

In [15]:
if(generate_incidents_csv):
    print("Generating incidents csv")
    df_incidents_info = df_traffic_reduced[df_traffic_reduced["accident_label"]][["incident_edge", "incident_type", 'incident_start_time',"accident_duration", "incident_lane"]]

    df_incidents_info2 = df_incidents_info.groupby("incident_start_time").first().reset_index()

    print("Getting head of incidents dataset")

    print(df_incidents_info2.head())


    df_incidents_info2.to_csv("experiments/E1/incidents_data.csv")

    print(f"Number of incidents in this simulation {len(df_incidents_info2)}")
    print(f"Number of incidents in this simulation with less severe incidents {len(df_incidents_info2[df_incidents_info2['incident_type']=='stalled_vehicle'])}")
    print(f"Number of incidents in this simulation with more severe incidents {len(df_incidents_info2[df_incidents_info2['incident_type']=='multi_vehicle_collision'])}")

    print(f"Input traffic raw data shape is {df_traffic.shape}")

Generating incidents csv
Getting head of incidents dataset
   incident_start_time incident_edge            incident_type  \
0               8140.0     436943781  multi_vehicle_collision   
1              15884.0   436794673#0  multi_vehicle_collision   
2              23462.0    -436789319  multi_vehicle_collision   
3              30662.0   436943731#0  multi_vehicle_collision   
4              38444.0     436943780  multi_vehicle_collision   

   accident_duration incident_lane  
0             6873.0   436943781_2  
1             3167.0             2  
2             7022.0             2  
3             6635.0             2  
4             5433.0             2  
Number of incidents in this simulation 100
Number of incidents in this simulation with less severe incidents 48
Number of incidents in this simulation with more severe incidents 52
Input traffic raw data shape is (82944000, 18)


In [16]:
# df_traffic_reduced = df_traffic_reduced.drop(["vehicles_per_lane_1","vehicles_per_lane_0","lane_mean_speed_0","lane_mean_speed_1"],axis=1)

In [17]:

if NUM_OF_JUNCTIONS == 3:
    print(f" Number of junctions considered 3")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","436943762#0"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2"]
    
elif NUM_OF_JUNCTIONS == 4:
    print(f" Number of junctions considered 4")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767"]
    
elif NUM_OF_JUNCTIONS == 5:
    print(f" Number of junctions considered 5")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0","436794669","436942357"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767","-436794676#1","531969915#0"]
    
elif NUM_OF_JUNCTIONS == 6:
    print(f" Number of junctions considered 6")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0","436794669","436942357","-436942356#1","436942362#0","436789580#1"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767","-436794676#1","531969915#0","436942356#0","-436942362#3","436789564#0"]
    
elif NUM_OF_JUNCTIONS == 7:
    print(f" Number of junctions considered 7")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0","436794669","436942357","-436942356#1","436942362#0","436789580#1","436794670","1051038541#0","-436794679#3"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767","-436794676#1","531969915#0","436942356#0","-436942362#3","436789564#0","-1088637809#1","436942385#0","436794679#0"]
    
elif NUM_OF_JUNCTIONS == 8:
    print(f" Number of junctions considered 8")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0","436794669","436942357","-436942356#1","436942362#0","436789580#1","436794670","1051038541#0","-436794679#3","436943745#0","351673438","-613687451#1"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767","-436794676#1","531969915#0","436942356#0","-436942362#3","436789564#0","-1088637809#1","436942385#0","436794679#0","-436943745#2","436943742","436943774"]
    

else:
    print(f" Number of junctions considered {NUM_OF_JUNCTIONS}")
    print("Entered too many sensors resorting to 8 sensors")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0","436794669","436942357","-436942356#1","436942362#0","436789580#1","436794670","1051038541#0","-436794679#3","436943745#0","351673438","-613687451#1"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767","-436794676#1","531969915#0","436942356#0","-436942362#3","436789564#0","-1088637809#1","436942385#0","436794679#0","-436943745#2","436943742","436943774"]
    

 Number of junctions considered 8


In [18]:
edges = incoming_edges + outgoing_edges

In [19]:
count = 0
df = df_traffic_reduced[df_traffic["identified_edge"] == edges[0] ]
df = df.reset_index(drop=True)
df[f"rolling_junction_mean_speed_{count}"] = df['junction_mean_speed'].rolling(ROLLING_WINDOW_LENGTH).mean()
df[f"rolling_traffic_occupancy_{count}"] = df['traffic_occupancy'].rolling(ROLLING_WINDOW_LENGTH).mean()
df[f"rolling_traffic_count_{count}"] = df['traffic_count'].rolling(ROLLING_WINDOW_LENGTH).mean()

Y = df[["step","time_of_day","incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label"]]
df_data = df[["step",f"rolling_junction_mean_speed_{count}",f"rolling_traffic_count_{count}",f"rolling_traffic_occupancy_{count}"]]
count+=1

/tmp/ipykernel_674921/770496982.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df_traffic_reduced[df_traffic["identified_edge"] == edges[0] ]


In [20]:
a = df.shape

print("Shape so far",a[0],a[1])

Shape so far 1296000 21


In [21]:
for index,edge in enumerate(edges):
    
    if index == 0:
        continue
    print(f"Processed {count}")
    df = df_traffic_reduced[df_traffic_reduced["identified_edge"] == edge ]
    df = df.reset_index(drop=True)
    df[f"rolling_junction_mean_speed_{count}"] = df['junction_mean_speed'].rolling(ROLLING_WINDOW_LENGTH).mean()
    df[f"rolling_traffic_count_{count}"] = df['traffic_count'].rolling(ROLLING_WINDOW_LENGTH).mean()
    df[f"rolling_traffic_occupancy_{count}"] = df['traffic_occupancy'].rolling(ROLLING_WINDOW_LENGTH).mean()
    
    df_temp = df[["step",f"rolling_junction_mean_speed_{count}",f"rolling_traffic_count_{count}",f"rolling_traffic_occupancy_{count}"]]
    df_data = pd.merge(left = df_data, right = df_temp , on = "step", how = "inner")

    count+=1
    

Processed 1
Processed 2
Processed 3
Processed 4
Processed 5
Processed 6
Processed 7
Processed 8
Processed 9
Processed 10
Processed 11
Processed 12
Processed 13
Processed 14
Processed 15
Processed 16
Processed 17
Processed 18
Processed 19
Processed 20
Processed 21
Processed 22
Processed 23
Processed 24
Processed 25
Processed 26
Processed 27
Processed 28
Processed 29
Processed 30
Processed 31
Processed 32
Processed 33
Processed 34
Processed 35
Processed 36
Processed 37
Processed 38
Processed 39
Processed 40
Processed 41


In [22]:
df_traffic_data = pd.merge(left = df_data, right = Y , on = "step", how = "inner")

In [23]:
for i in df_traffic_data.columns:
    print(i)

step
rolling_junction_mean_speed_0
rolling_traffic_count_0
rolling_traffic_occupancy_0
rolling_junction_mean_speed_1
rolling_traffic_count_1
rolling_traffic_occupancy_1
rolling_junction_mean_speed_2
rolling_traffic_count_2
rolling_traffic_occupancy_2
rolling_junction_mean_speed_3
rolling_traffic_count_3
rolling_traffic_occupancy_3
rolling_junction_mean_speed_4
rolling_traffic_count_4
rolling_traffic_occupancy_4
rolling_junction_mean_speed_5
rolling_traffic_count_5
rolling_traffic_occupancy_5
rolling_junction_mean_speed_6
rolling_traffic_count_6
rolling_traffic_occupancy_6
rolling_junction_mean_speed_7
rolling_traffic_count_7
rolling_traffic_occupancy_7
rolling_junction_mean_speed_8
rolling_traffic_count_8
rolling_traffic_occupancy_8
rolling_junction_mean_speed_9
rolling_traffic_count_9
rolling_traffic_occupancy_9
rolling_junction_mean_speed_10
rolling_traffic_count_10
rolling_traffic_occupancy_10
rolling_junction_mean_speed_11
rolling_traffic_count_11
rolling_traffic_occupancy_11
rolli

In [24]:
a = df_traffic_data.shape

print("Shape so far",a[0],a[1])

Shape so far 1296000 135


In [25]:
# Dealing with Vehicle data
dtype = {
'step':'int64',
'time_of_day':'int64',
'car_id':'object',
'identified_edge':'object',
'identified_lane':'float64',
'junction_mean_speed':'float64',
'vehicle_speed':'float64',
'vehicle_acceleration':'float64'
}
df_vehicle = pd.read_csv(eval_vehicle,dtype=dtype)

In [26]:
df_vehicle.head()

,step,time_of_day,car_id,identified_edge,identified_lane,vehicle_speed,vehicle_acceleration,junction
0,7,7,veh7,-436942356#1,1.0,0.000000,0.000000,6
1,7,7,veh7.1,-436942356#1,2.0,0.000000,0.000000,6
2,8,8,veh7,-436942356#1,1.0,1.433850,1.433850,6
3,8,8,veh7.1,-436942356#1,2.0,2.020224,2.020224,6
4,9,9,veh7,-436942356#1,1.0,4.033344,2.599494,6


In [27]:
df_vehicle = df_vehicle[~df_vehicle['identified_edge'].isin(excess_list)]

In [28]:
df_vehicle.head()

,step,time_of_day,car_id,identified_edge,identified_lane,vehicle_speed,vehicle_acceleration,junction
0,7,7,veh7,-436942356#1,1.0,0.000000,0.000000,6
1,7,7,veh7.1,-436942356#1,2.0,0.000000,0.000000,6
2,8,8,veh7,-436942356#1,1.0,1.433850,1.433850,6
3,8,8,veh7.1,-436942356#1,2.0,2.020224,2.020224,6
4,9,9,veh7,-436942356#1,1.0,4.033344,2.599494,6


In [29]:
df_vehicle = df_vehicle.drop(["junction"],axis=1)

In [30]:
# sensor details

sensors = {

    1: "5607328#0",
    2:"436791113#0",
    3:"533573776#0",
    4:"436794680#0",
    5:"436794670",
    6:"436942385#0",
    7:"1051038541#0",
    8:"436794679#0",
    9:"-436794679#3",
    10:"-1088637809#1",
    11:"436794669",
    12:"531969915#0",
    13:"436942357",
    14:"-436794676#1",
    15:"436790495",
    16:"533371302#0",
    17:"-1033824750",
    18:"436942374",
    19:"30031286#0",
    20:"436790491",
    21:"436942381#0",
    22:"-436942381#3",
    23:"436789580#1",
    24:"436942356#0",
    25:"-436942356#1",
    26:"-436942362#3",
    27:"436942362#0",
    28:"436789564#0",
    29:"436940278",
    30:"436940270",
    31:"-643913497",
    32:"519448767",
    33:"436943745#0",
    34:"436943742",
    35:"351673438",
    36:"436943774",
    37:"-613687451#1",
    38:"-436943745#2",
    39:"436943750#0",
    40:"-436943762#2",
    41:"436943762#0",
    42:"436943743#0"

}

In [31]:
junctions_sensor_combo ={

    3:[[3,22],[21,4],[1,16],[15,2],[40,20],[19,41],[42,18],[17,39],[3,41],[42,2],[1,39],[40,4]],

    4:[[3,22],[21,4],[1,16],[15,2],[40,20],[19,41],[42,18],[17,39],[3,41],[42,31],[30,39],[40,4],[32,2],[1,29],[32,16],[15,29],[30,18],[17,31]],
    
    5:[[3,11],[14,4],[30,39],[42,31],[32,2],[1,29],[40,13],[12,41],[3,22],[14,22],[21,4],[21,11],[1,16],[15,2],[19,13],[12,20],[32,16],[15,29],[30,18],[17,31],[17,39],[42,18],[19,41],[40,20]],

    6:[[3,11],[14,4],[30,39],[42,31],[32,2],[1,29],[26,13],[12,27],[40,25],[24,41],[3,22],[14,22],[21,4],[21,11],[1,16],[15,2],[19,23],[28,20],[32,16],[15,29],[30,18],[17,31],[17,39],[42,18]],

    7:[[3,5],[10,4],[8,11],[14,9],[30,39],[42,31],[32,2],[1,29],[26,13],[12,27],[40,25],[24,41],[6,22],[21,7],[1,16],[15,2],[19,23],[28,20],[32,16],[15,29],[30,18],[17,31],[17,39],[42,18]],

    8:[[3,5],[10,4],[8,11],[14,9],[30,33],[38,31],[34,39],[42,35],[32,2],[1,29],[26,13],[12,27],[40,25],[24,41],[6,22],[21,7],[1,16],[15,2],[19,23],[28,20],[32,16],[15,29],[36,18],[17,37]]

}

In [32]:
interested_combo = junctions_sensor_combo[NUM_OF_JUNCTIONS]

In [33]:
print(interested_combo)
print(len(interested_combo))

[[3, 5], [10, 4], [8, 11], [14, 9], [30, 33], [38, 31], [34, 39], [42, 35], [32, 2], [1, 29], [26, 13], [12, 27], [40, 25], [24, 41], [6, 22], [21, 7], [1, 16], [15, 2], [19, 23], [28, 20], [32, 16], [15, 29], [36, 18], [17, 37]]
24


In [34]:
steps = [i for i in range(LENGTH_OF_SIM)]
data_dd = {"steps":steps}
dd_dataframe = pd.DataFrame(data_dd)
dd_dataframe.shape

(1296000, 1)

In [35]:
count = 1
for sensor_combination in interested_combo:
    print(count)
    outgoing_sensor = sensors[sensor_combination[0]]
    incoming_sensor = sensors[sensor_combination[1]]
    df_outgoing = df_vehicle[df_vehicle['identified_edge'] == outgoing_sensor]
    df_incoming = df_vehicle[df_vehicle['identified_edge'] == incoming_sensor]
    df_incoming = df_incoming.reset_index(drop=True)
    df_outgoing = df_outgoing.reset_index(drop=True)

    df_outgoing_unique = df_outgoing[df_outgoing["car_id"].notnull()]
    df_incoming_unique = df_incoming[df_incoming["car_id"].notnull()]

    df_outgoing_unique = df_outgoing_unique.groupby('car_id').first().reset_index()
    df_incoming_unique = df_incoming_unique.groupby('car_id').first().reset_index()

    df_outgoing_unique = df_outgoing_unique.sort_values(by = 'step')
    df_incoming_unique = df_incoming_unique.sort_values(by = 'step')
    
    df_merged = pd.merge(left = df_incoming_unique,right = df_outgoing_unique, on='car_id', how='inner')

    df_merged["travel_time"] = df_merged["step_x"] - df_merged["step_y"]

    df_travel_time_data = df_merged[["step_x","travel_time"]]
    df_travel_time_data = df_travel_time_data.groupby("step_x").mean().reset_index()
    
    df_travel_time_data2 = pd.merge(left = df_travel_time_data,right = dd_dataframe, right_on='steps',left_on="step_x", how='right')

    df_travel_time_data2[f"rolling_travel_time_{sensor_combination[0]}_{sensor_combination[1]}"] = df_travel_time_data2["travel_time"].rolling(300,min_periods=1).mean()

    df_travel_time_data2 = df_travel_time_data2.drop(["step_x","travel_time"],axis=1)
    df_traffic_data = pd.merge(left = df_traffic_data, right = df_travel_time_data2 , left_on = "step",right_on = "steps", how = "inner")
    df_traffic_data = df_traffic_data.drop(["steps"],axis=1)
    count+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [36]:
df_traffic_data.shape

(1296000, 159)

In [37]:
df_traffic_data.head()

,step,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,rolling_junction_mean_speed_2,rolling_traffic_count_2,rolling_traffic_occupancy_2,...,rolling_travel_time_6_22,rolling_travel_time_21_7,rolling_travel_time_1_16,rolling_travel_time_15_2,rolling_travel_time_19_23,rolling_travel_time_28_20,rolling_travel_time_32_16,rolling_travel_time_15_29,rolling_travel_time_36_18,rolling_travel_time_17_37
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df = df_traffic_data[600:]
Y = df["accident_label"]
X = df.drop(["incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label"],axis=1)

In [39]:
def standardize(col):
   return (col - col.mean()) / col.std()

X = X.apply(standardize)

In [40]:
# file_name = "/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/xgboost/saved_models/V4/xg_model.pkl"
xgb_model_loaded = pickle.load(open(model_path, "rb"))

In [41]:
X = X.drop(["step"],axis=1)

In [42]:
X.head(500)

,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,rolling_junction_mean_speed_2,rolling_traffic_count_2,rolling_traffic_occupancy_2,rolling_junction_mean_speed_3,...,rolling_travel_time_6_22,rolling_travel_time_21_7,rolling_travel_time_1_16,rolling_travel_time_15_2,rolling_travel_time_19_23,rolling_travel_time_28_20,rolling_travel_time_32_16,rolling_travel_time_15_29,rolling_travel_time_36_18,rolling_travel_time_17_37
600,-2.677477,-0.786082,-0.790246,-2.474267,-0.965732,-0.991840,-1.109187,-0.575572,-0.580663,-2.238416,...,-0.147376,-0.366140,-0.270165,-0.144184,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
601,-2.677477,-0.786082,-0.790246,-2.474267,-0.965732,-0.991840,-1.091619,-0.574702,-0.579749,-2.232221,...,-0.147376,-0.366140,-0.270165,-0.137242,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
602,-2.677477,-0.786082,-0.790246,-2.439902,-0.964398,-0.990416,-1.074051,-0.573833,-0.578835,-2.226026,...,-0.147376,-0.366140,-0.270165,-0.137242,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
603,-2.677477,-0.786082,-0.790246,-2.407970,-0.963064,-0.988993,-1.056483,-0.572963,-0.577921,-2.219830,...,-0.147376,-0.366140,-0.270165,-0.130835,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
604,-2.677477,-0.786082,-0.790246,-2.378800,-0.960396,-0.986147,-1.038916,-0.572094,-0.577008,-2.213635,...,-0.147376,-0.366140,-0.270165,-0.130835,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,-1.861819,-0.484175,-0.489655,-0.269659,0.652458,0.632856,1.596943,-0.485152,-0.486696,-0.125154,...,-0.195075,-0.064349,-0.394550,0.151681,0.074850,-0.385281,NaN,NaN,-0.401017,-0.337494
1096,-1.855056,-0.475347,-0.480887,-0.287047,0.660463,0.640331,1.592904,-0.480805,-0.482552,-0.109524,...,-0.187844,-0.064349,-0.398752,0.151681,0.074850,-0.385281,NaN,NaN,-0.401017,-0.337494
1097,-1.850114,-0.466520,-0.472118,-0.302319,0.667133,0.646964,1.588863,-0.476458,-0.478409,-0.093855,...,-0.185685,-0.064349,-0.398752,0.151681,0.074850,-0.385281,NaN,NaN,-0.401017,-0.337494
1098,-1.846418,-0.457692,-0.463350,-0.314793,0.673803,0.654079,1.586276,-0.473850,-0.474644,-0.077411,...,-0.183965,-0.064349,-0.398752,0.151681,0.074850,-0.385281,NaN,NaN,-0.401017,-0.337494


In [43]:
for col in X.columns:
    
    print(col)

rolling_junction_mean_speed_0
rolling_traffic_count_0
rolling_traffic_occupancy_0
rolling_junction_mean_speed_1
rolling_traffic_count_1
rolling_traffic_occupancy_1
rolling_junction_mean_speed_2
rolling_traffic_count_2
rolling_traffic_occupancy_2
rolling_junction_mean_speed_3
rolling_traffic_count_3
rolling_traffic_occupancy_3
rolling_junction_mean_speed_4
rolling_traffic_count_4
rolling_traffic_occupancy_4
rolling_junction_mean_speed_5
rolling_traffic_count_5
rolling_traffic_occupancy_5
rolling_junction_mean_speed_6
rolling_traffic_count_6
rolling_traffic_occupancy_6
rolling_junction_mean_speed_7
rolling_traffic_count_7
rolling_traffic_occupancy_7
rolling_junction_mean_speed_8
rolling_traffic_count_8
rolling_traffic_occupancy_8
rolling_junction_mean_speed_9
rolling_traffic_count_9
rolling_traffic_occupancy_9
rolling_junction_mean_speed_10
rolling_traffic_count_10
rolling_traffic_occupancy_10
rolling_junction_mean_speed_11
rolling_traffic_count_11
rolling_traffic_occupancy_11
rolling_ju

In [44]:
X.head(20)

,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,rolling_junction_mean_speed_2,rolling_traffic_count_2,rolling_traffic_occupancy_2,rolling_junction_mean_speed_3,...,rolling_travel_time_6_22,rolling_travel_time_21_7,rolling_travel_time_1_16,rolling_travel_time_15_2,rolling_travel_time_19_23,rolling_travel_time_28_20,rolling_travel_time_32_16,rolling_travel_time_15_29,rolling_travel_time_36_18,rolling_travel_time_17_37
600,-2.677477,-0.786082,-0.790246,-2.474267,-0.965732,-0.991840,-1.109187,-0.575572,-0.580663,-2.238416,...,-0.147376,-0.366140,-0.270165,-0.144184,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
601,-2.677477,-0.786082,-0.790246,-2.474267,-0.965732,-0.991840,-1.091619,-0.574702,-0.579749,-2.232221,...,-0.147376,-0.366140,-0.270165,-0.137242,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
602,-2.677477,-0.786082,-0.790246,-2.439902,-0.964398,-0.990416,-1.074051,-0.573833,-0.578835,-2.226026,...,-0.147376,-0.366140,-0.270165,-0.137242,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
603,-2.677477,-0.786082,-0.790246,-2.407970,-0.963064,-0.988993,-1.056483,-0.572963,-0.577921,-2.219830,...,-0.147376,-0.366140,-0.270165,-0.130835,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
604,-2.677477,-0.786082,-0.790246,-2.378800,-0.960396,-0.986147,-1.038916,-0.572094,-0.577008,-2.213635,...,-0.147376,-0.366140,-0.270165,-0.130835,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
605,-2.677477,-0.786082,-0.790246,-2.353090,-0.957728,-0.983301,-1.021348,-0.571225,-0.576094,-2.207440,...,-0.147376,-0.366140,-0.270165,-0.130835,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
606,-2.677477,-0.786082,-0.790246,-2.329899,-0.955060,-0.980455,-1.003780,-0.570355,-0.575180,-2.201245,...,-0.147376,-0.366140,-0.270165,-0.130835,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
607,-2.677477,-0.786082,-0.790246,-2.308516,-0.952392,-0.977609,-0.986212,-0.569486,-0.574266,-2.195050,...,-0.147376,-0.366140,-0.270165,-0.130835,0.086343,-0.336872,NaN,NaN,-0.548096,0.097430
608,-2.677477,-0.786082,-0.790246,-2.287250,-0.949724,-0.974763,-0.968644,-0.568616,-0.573352,-2.188854,...,-0.147376,-0.366140,-0.270165,-0.130835,0.086343,-0.336872,NaN,NaN,-0.531606,0.097430
609,-2.677477,-0.786082,-0.790246,-2.265989,-0.947056,-0.971917,-0.951076,-0.567747,-0.572438,-2.182659,...,-0.147376,-0.366140,-0.270165,-0.130835,0.086343,-0.336872,NaN,NaN,-0.531606,0.097430


In [45]:
import numpy as np
def callme(prediction_rate):
    history = [0]  # Assuming no incident at start
    incident_counter = 0
    detected_counter = 0
    new_incident_flag = False
    incident_detected_flag = False
    predictions_count = 0
    i = 0  # Time since last new incident
    false_alarm_counter = 0
    detection_times = []  # To calculate mean time for detection
    counter = 1
    for (index,row),(yrow) in zip(X.iterrows(),Y.to_numpy()):
        
        if counter% prediction_rate == 0:
            
            predictions_count+=1
            predicted = xgb_model_loaded.predict([row.values])[0] 
            # print(predicted)
            
            if len(history) > 0:
                previous_label = history[-1]
            else:
                previous_label = 0  # Default to no incident for the first element

            # Update history
            current_label = 1 if yrow else 0
            history.append(current_label)

            # New incident started
            if previous_label == 0 and current_label == 1:
                incident_counter += 1
                new_incident_flag = True
                incident_detected_flag = False  # Reset detection flag for new incident
                i = 0  # Reset timer for new incident
                print("New incident started")

            # Incident detected
            if new_incident_flag and predicted == 1 and not incident_detected_flag:
                detected_counter += 1
                detection_times.append(i)
                incident_detected_flag = True
                print(f"Incident Detected by model after {i} timesteps.")

            # Incident finished
            if previous_label == 1 and current_label == 0:
                new_incident_flag = False
                print("Incident Finished")


            # False alarm raised
            if not new_incident_flag and predicted == 1 and not (1 in history[-1000:]):
                false_alarm_counter += 1
                
        # Increment timer if incident ongoing but not yet detected
        if new_incident_flag and not incident_detected_flag:
            i += 1
        counter+=1
        
        
    arr = np.array(detection_times)
    row = {"prediction_rate" : prediction_rate, 
           "incident_counter": incident_counter,
           "detected_counter":detected_counter,
           "mttd":arr.mean(),
           "false_alarm_counter":false_alarm_counter,
           "predictions_count":predictions_count
        
    }    
    return row
                # print("False Alarm raised by model.")

In [46]:
# incident_counter = 0
# detected_counter = 0
# new_incident_flag = False
# incident_detected_flag = False
# predictions_count = 0
# i = 0  # Time since last new incident
# false_alarm_counter = 0
# detection_times = []  # To calculate mean time for detection
import numpy as np
rows = []
experiments = [30]

for i in experiments:
    print("Running experiment on ",i)
    rows.append(callme(i))

Running experiment on  30
New incident started
Incident Detected by model after 0 timesteps.
Incident Finished
New incident started
Incident Detected by model after 180 timesteps.
Incident Finished
New incident started
Incident Detected by model after 0 timesteps.
Incident Finished
New incident started
Incident Detected by model after 0 timesteps.
Incident Finished
New incident started
Incident Detected by model after 0 timesteps.
Incident Finished
New incident started
Incident Detected by model after 30 timesteps.
Incident Finished
New incident started
Incident Detected by model after 120 timesteps.
Incident Finished
New incident started
Incident Detected by model after 0 timesteps.
Incident Finished
New incident started
Incident Detected by model after 30 timesteps.
Incident Finished
New incident started
Incident Detected by model after 240 timesteps.
Incident Finished
New incident started
Incident Detected by model after 0 timesteps.
Incident Finished
New incident started
Incident D

In [47]:
results = pd.DataFrame(rows)

In [48]:
results["false_alarm_rate"] = 100*results["false_alarm_counter"]/results["predictions_count"]
results["accuracy"] = 100*results["detected_counter"]/results["incident_counter"]

In [49]:
results.head()
# 5 junctions

,prediction_rate,incident_counter,detected_counter,mttd,false_alarm_counter,predictions_count,false_alarm_rate,accuracy
0,30,100,100,94.5,4764,43180,11.032886,100.0
